In [4]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import KFold, train_test_split, GridSearchCV

In [1]:
%store -r y_pred

In [5]:
file = open('test_file_trees.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_pred)):
    writer.writerow([i, y_pred[i]])
file.close()